# 🎵 HeartMuLa 3B Music Generator - Free Tier Optimized
### Created by AIQUEST (@AIQuestAcademy)

---

## 📖 About This Notebook

This notebook allows you to run **HeartMuLa 3B**, an open-source AI music generation model, on **Google Colab's FREE tier** using BF16 optimization and lazy loading techniques.

### ✨ Key Features
- 🆓 **Free Tier Compatible** - Runs on T4 GPU (15GB VRAM)
- 🎼 **Professional Quality** - Generate music with lyrics and style control
- ⚡ **BF16 Optimized** - 50% memory reduction vs full precision
- 🎚️ **Advanced Controls** - Temperature, Top-K, CFG Scale adjustments
- 🌍 **Multilingual Support** - Generate music in multiple languages
- ⏱️ **Time Estimate** - ~5-7 minutes per minute of audio

### 📊 Technical Specs
- **Model:** HeartMuLa-oss-3B (BF16)
- **Components:** HeartCodec, HeartMuLaGen, HeartTranscriptor
- **Framework:** PyTorch + Gradio
- **GPU Required:** NVIDIA T4 or better (15GB+ VRAM)

### 🎬 Support AIQUEST
If you find this notebook useful, please support my work:
- 🔴 **YouTube:** [@AIQuestAcademy](https://youtube.com/@AIQuestAcademy)
- 🐦 **X/Twitter:** [@AIQuestAcademy](https://twitter.com/AIQuestAcademy)

I spend hours optimizing and testing these models so you don't have to!

### ⚠️ Important Notes
- **Generation Time:** Expect 5-7 minutes per minute of audio
- **Colab Timeout:** Free tier sessions timeout after 90 minutes of inactivity
- **First Run:** Initial model loading may take extra time

### 📚 Credits
- **Original Model:** [HeartMuLa Team](https://github.com/HeartMuLa/heartlib)
- **Research Paper:** [arXiv:2601.10547](https://arxiv.org/abs/2601.10547)
- **BF16 Checkpoints:** [benjiaiplayground](https://huggingface.co/benjiaiplayground)
- **Notebook Optimization:** AIQUEST (@AIQuestAcademy)

---

## 🚀 Quick Start Guide

1. **Run Cell 1:** Install dependencies and setup environment (~3-5 minutes)
2. **Run Cell 2:** Download BF16 models (~5-10 minutes, one-time only)
3. **Run Cell 3:** Launch Gradio interface and start generating music!

**Ready? Let's make some music! 🎵**

---


In [ ]:
# @title Setup Environment
import os

print("🔧 Setting up environment...")
print("="*60)

# Step 1: Install system dependencies
print("\n[1/5] 📦 Installing FFmpeg...")
!sudo apt-get update -qq && sudo apt-get install -y ffmpeg > /dev/null 2>&1
print("✓ FFmpeg installed")

# Step 2: Clone repository (if not already cloned)
print("\n[2/5] 📥 Cloning HeartMuLa repository...")
if not os.path.exists("/content/heartlib"):
    !git clone https://github.com/HeartMuLa/heartlib.git /content/heartlib -q
    print("✓ Repository cloned")
else:
    print("✓ Repository already exists")

# Step 3: Change to heartlib directory BEFORE installing
%cd /content/heartlib

# Step 4: Install Python packages in correct order
print("\n[3/5] 📦 Installing huggingface_hub...")
!pip install -q "huggingface_hub>=1.3.0,<2.0"
print("✓ huggingface_hub installed")

print("\n[4/5] 📦 Installing HeartMuLa package...")
!pip install -q -e .
print("✓ HeartMuLa package installed")

print("\n[5/5] 📦 Installing additional dependencies...")
!pip install -q flash-attn --no-build-isolation
!pip install -q accelerate
print("✓ Additional dependencies installed")

print("\n" + "="*60)
print("✅ Environment setup complete!")
print("="*60)

In [ ]:
# @title Download Models (BF16 Optimized)
import os
import shutil

# Ensure we're in the heartlib directory
if os.path.basename(os.getcwd()) != "heartlib":
    %cd /content/heartlib

os.makedirs('./ckpt', exist_ok=True)

print("="*70)
print("📥 DOWNLOADING HEARTMULA MODELS (BF16 OPTIMIZED)")
print("="*70)

# Model 1: HeartMuLa-oss-3B
print("\n[1/3] 📦 HeartMuLa-oss-3B (Main Generation Model)")
!huggingface-cli download --local-dir './ckpt/HeartMuLa-oss-3B' 'benjiaiplayground/HeartMuLa-oss-3B-bf16' --quiet
print("✓ Downloaded")

# Model 2: HeartCodec-oss
print("\n[2/3] 📦 HeartCodec-oss (Audio Codec)")
!huggingface-cli download --local-dir './ckpt/HeartCodec-oss' 'benjiaiplayground/HeartCodec-oss-bf16' --quiet
print("✓ Downloaded")

# Model 3: HeartMuLaGen (required for gen_config.json and tokenizer.json)
print("\n[3/3] 📦 HeartMuLaGen (Config & Tokenizer)")
!huggingface-cli download --local-dir './ckpt' 'HeartMuLa/HeartMuLaGen' --quiet
print("✓ Downloaded")

# Fix HeartCodec file naming
print("\n" + "="*70)
print("⚙️  CONFIGURING MODEL FILES...")
codec_file = './ckpt/HeartCodec-oss/HeartCodec-oss-bf16.safetensors'
if os.path.exists(codec_file):
    shutil.move(codec_file, './ckpt/HeartCodec-oss/model.safetensors')
    print("✓ HeartCodec checkpoint configured")
else:
    print("ℹ️ HeartCodec already configured or using different naming")

print("\n" + "="*70)
print("✅ ALL MODELS DOWNLOADED SUCCESSFULLY!")
print("="*70)

# Verify structure
print("\n📁 Checkpoint structure:")
!ls -la ./ckpt/

In [ ]:
# @title Launch Music Generator Interface
import gradio as gr
import os
import subprocess
import time
from datetime import datetime

# Ensure we're in the heartlib directory
if os.path.basename(os.getcwd()) != "heartlib":
    if os.path.exists("/content/heartlib"):
        os.chdir("/content/heartlib")

def generate_music(lyrics, tags, duration, temperature, topk, cfg_scale, progress=gr.Progress()):
    """Generate music with advanced controls"""
    try:
        start_time = time.time()
        progress(0, desc="🎵 Initializing generation...")

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_path = f"./assets/music_{timestamp}.mp3"
        os.makedirs("./assets", exist_ok=True)

        # Write lyrics to a temporary file (script expects file paths, not raw text)
        lyrics_file = f"./assets/lyrics_{timestamp}.txt"
        with open(lyrics_file, "w", encoding="utf-8") as f:
            f.write(lyrics)

        # Write tags to a temporary file
        tags_file = f"./assets/tags_{timestamp}.txt"
        with open(tags_file, "w", encoding="utf-8") as f:
            f.write(tags)

        duration_ms = int(duration * 1000)

        # Estimate generation time (5-7 minutes per minute of audio)
        estimated_minutes = (duration / 60) * 6  # Average 6 minutes per minute of audio

        cmd = [
            "python", "./examples/run_music_generation.py",
            f"--model_path=./ckpt",
            f"--version=3B",
            f"--lazy_load=true",
            f"--lyrics={lyrics_file}",
            f"--tags={tags_file}",
            f"--max_audio_length_ms={duration_ms}",
            f"--temperature={temperature}",
            f"--topk={topk}",
            f"--cfg_scale={cfg_scale}",
            f"--save_path={output_path}"
        ]

        progress(0.1, desc=f"⏱️ Estimated time: ~{estimated_minutes:.0f} minutes")
        progress(0.2, desc="🎼 Processing lyrics and tags...")
        progress(0.3, desc="🎹 Generating music tokens...")

        result = subprocess.run(cmd, capture_output=True, text=True)
        elapsed_time = time.time() - start_time

        # Clean up temp files
        try:
            os.remove(lyrics_file)
            os.remove(tags_file)
        except:
            pass

        if result.returncode == 0 and os.path.exists(output_path):
            progress(1.0, desc="✅ Generation complete!")
            stats = f"""
✅ **Generation Complete!**
- 🎵 Audio Duration: {duration}s ({duration/60:.1f} min)
- ⏱️ Generation Time: {elapsed_time/60:.1f} minutes
- 🎚️ Settings: Temp={temperature} | TopK={topk} | CFG={cfg_scale}
            """
            return output_path, stats
        else:
            error_msg = result.stderr if result.stderr else result.stdout
            return None, f"❌ Generation Error:\n```\n{error_msg[-1000:] if len(error_msg) > 1000 else error_msg}\n```"
    except Exception as e:
        return None, f"❌ Error: {str(e)}"

# Custom CSS for better button styling
custom_css = """
.youtube-btn {
    background: linear-gradient(135deg, #FF0000 0%, #CC0000 100%) !important;
    color: white !important;
    font-size: 16px !important;
    font-weight: bold !important;
    padding: 12px 24px !important;
    border: none !important;
    border-radius: 8px !important;
    box-shadow: 0 4px 6px rgba(0,0,0,0.1) !important;
}
.youtube-btn:hover {
    transform: translateY(-2px);
    box-shadow: 0 6px 8px rgba(0,0,0,0.15) !important;
}
.twitter-btn {
    background: linear-gradient(135deg, #1DA1F2 0%, #0C85D0 100%) !important;
    color: white !important;
    font-size: 16px !important;
    font-weight: bold !important;
    padding: 12px 24px !important;
    border: none !important;
    border-radius: 8px !important;
    box-shadow: 0 4px 6px rgba(0,0,0,0.1) !important;
}
.twitter-btn:hover {
    transform: translateY(-2px);
    box-shadow: 0 6px 8px rgba(0,0,0,0.15) !important;
}
"""

# Create interface
with gr.Blocks(theme=gr.themes.Soft(), css=custom_css, title="🎵 HeartMuLa 3B by AIQUEST") as demo:

    gr.Markdown("""
    # 🎵 HeartMuLa 3B by AIQUEST
    ### Free & Open Source AI Music Generation on Google Colab Free Tier
    """)

    # Support message with styled buttons
    gr.Markdown("""
    ### 💝 Support This Work
    I spent hours optimizing HeartMuLa 3B with BF16 and lazy loading so you can generate professional music on Colab's free tier.
    **If you find this helpful, please support by:**
    """)

    with gr.Row():
        youtube_link = gr.Button(
            "🔴 SUBSCRIBE ON YOUTUBE",
            link="https://youtube.com/@AIQuestAcademy",
            elem_classes="youtube-btn",
            scale=1
        )
        twitter_link = gr.Button(
            "🐦 FOLLOW ON X/TWITTER",
            link="https://twitter.com/AIQuestAcademy",
            elem_classes="twitter-btn",
            scale=1
        )

    gr.Markdown("---")

    # Main interface
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 📝 Lyrics & Style")

            lyrics_input = gr.Textbox(
                label="Song Lyrics",
                placeholder="[Intro]\n...\n\n[Verse]\nYour lyrics here...\n\n[Chorus]\nMore lyrics...",
                lines=12,
                value="""[Intro]
Soft piano melody begins

[Verse]
The sun creeps in across the floor
I hear the traffic outside the door
The coffee pot begins to hiss
Another morning just like this

[Chorus]
Every day the light returns
Every day the fire burns"""
            )

            tags_input = gr.Textbox(
                label="Style Tags (comma-separated)",
                placeholder="piano, happy, pop",
                value="piano, calm, acoustic, morning"
            )

            gr.Markdown("### ⚙️ Advanced Settings")

            duration_slider = gr.Slider(
                30, 360, value=90, step=10,
                label="Duration (seconds)",
                info="Note: 1 min audio = 5-7 min generation time"
            )

            with gr.Row():
                temperature_slider = gr.Slider(
                    0.7, 1.3, value=1.0, step=0.1,
                    label="Temperature",
                    info="Higher = more creative"
                )
                topk_slider = gr.Slider(
                    20, 100, value=50, step=5,
                    label="Top-K",
                    info="Diversity control"
                )

            cfg_scale_slider = gr.Slider(
                1.0, 3.0, value=1.5, step=0.1,
                label="CFG Scale",
                info="Style adherence"
            )

            generate_btn = gr.Button("🎵 Generate Music", variant="primary", size="lg")

        with gr.Column(scale=1):
            gr.Markdown("### 🎧 Generated Output")
            output_audio = gr.Audio(label="Your Music", type="filepath")
            stats_output = gr.Markdown("ℹ️ Click Generate to start...")

            gr.Markdown("""
            ### 💡 Generation Time Guide
            - **30s audio:** ~2-3 minutes
            - **60s audio:** ~5-7 minutes
            - **90s audio:** ~7-10 minutes
            - **120s audio:** ~10-14 minutes
            - **180s audio:** ~15-21 minutes

            ⚠️ **Colab sessions timeout after 90 minutes of inactivity**

            ### 🎼 Tips
            - Start with 60-90s for faster testing
            - Temperature 1.0 = balanced, 1.2+ = experimental
            - CFG 1.5-2.0 for strong style adherence
            - Use [Intro], [Verse], [Chorus], [Bridge], [Outro] tags
            """)

    # Examples
    gr.Markdown("### 🎼 Example Prompts (Click to Load)")
    gr.Examples(
        examples=[
            ["[Verse]\nNeon lights paint the street\nCity pulse beneath my feet\n\n[Chorus]\nAlive tonight, feeling right", "electronic, upbeat, synthwave, energetic", 90, 1.0, 50, 1.5],
            ["[Verse]\nQuiet morning light\nSoft and bright\nPeaceful moments here\n\n[Chorus]\nStay a while with me", "acoustic, calm, folk, gentle guitar", 60, 0.9, 40, 1.8],
            ["[Verse]\nThunder rolls across the sky\nLightning flashes way up high\n\n[Chorus]\nStorm is here, feel the power", "rock, heavy, electric guitar, drums", 120, 1.1, 60, 2.0],
            ["[Intro]\nJazz piano\n\n[Verse]\nSmooth nights in the lounge\nSax melody all around\n\n[Chorus]\nLost in rhythm", "jazz, saxophone, piano, lounge, smooth", 90, 1.0, 50, 1.5],
        ],
        inputs=[lyrics_input, tags_input, duration_slider, temperature_slider, topk_slider, cfg_scale_slider],
    )

    gr.Markdown("""
    ---
    ### 📚 Credits & Attribution

    **Original Model:** HeartMuLa by [HeartMuLa Team](https://github.com/HeartMuLa/heartlib) | [Research Paper](https://arxiv.org/abs/2601.10547)
    **BF16 Optimization:** [benjiaiplayground](https://huggingface.co/benjiaiplayground) on Hugging Face
    **Colab Implementation:** @AIQuestAcademy (BF16 + Lazy Loading for Free Tier)
    **Framework:** Gradio | **License:** Open Source

    This notebook uses memory-optimized BF16 models with lazy loading to fit within Google Colab's free tier T4 GPU (15GB VRAM).
    """)

    generate_btn.click(
        fn=generate_music,
        inputs=[lyrics_input, tags_input, duration_slider, temperature_slider, topk_slider, cfg_scale_slider],
        outputs=[output_audio, stats_output]
    )

print("\n" + "="*70)
print("🎵 HeartMuLa 3B by AIQUEST")
print("="*70)
demo.launch(share=True, debug=False)